In [1]:
import csv
import pandas as pd
import re
from datetime import datetime, timedelta
import numpy as np
from scipy.interpolate import griddata
import xarray as xr
import matplotlib.pyplot as plt
import os
from shapely.geometry import Point
import geopandas as gpd

# 风速

## 读取数据

In [11]:
# 地球的半径
R  = 6371

def read_spw_file(file_path):
    """Reads an spw file and returns a list of lists.

    Args:
        file_path: The path to the spw file.

    Returns:
        A list of lists, where each inner list represents a row in the spw file.
    """

    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        data = []
        for row in reader:
            data.append(row)

    return data

# 创建一个dataframe，包括x,y,t,u,direction, presure


data_9711 = []

if __name__ == '__main__':
    file_path = '../data/raw/9711.spw'
    data = read_spw_file(file_path)

    # Get the variables n_cols, n_rows, and spw_radius.
    n_cols = int(data[3][0].split('=')[1].split('#')[0].strip().replace(' ', ''))
    n_rows = int(data[4][0].split('=')[1].split('#')[0].strip().replace(' ', ''))
    spw_radius = int(data[6][0].split('=')[1].split('#')[0].strip().replace(' ', ''))
    
    match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', data[15][0])
    if match:
        original_time = datetime.strptime(match.group(), '%Y-%m-%d %H:%M:%S')
    else:
        raise ValueError("No match found for time value in data")
    
    loop_number = 4 + n_rows * 3
    # data 每4+500+500+500行循环读取一次
    for i in range(15, len(data)-loop_number*2, loop_number):
        # Process the data here
        rows = data[i:i+4+500*3]
        
        match = re.search(r'[-+]?\d*\.\d+', rows[0][0])
        if match:
            time = float(match.group())
            
            # Calculate the accurate time
            t = original_time + timedelta(minutes=time)
        else:
            raise ValueError

        match_x = re.search(r'x_spw_eye\s*=\s*([\d.]+)', rows[1][0])
        match_y = re.search(r'y_spw_eye\s*=\s*(\d+)', rows[2][0])

        if match_x:
            x_spw_eye = float(match_x.group(1))  # Convert the captured group to float
        else:
            raise ValueError("No match found for x_spw_eye value in data")

        if match_y:
            y_spw_eye = int(match_y.group(1))  # Convert the captured group to integer
        else:
            raise ValueError("No match found for y_spw_eye value in data")
            
        for i in range(0,200,10):
            for j in range(n_cols):
                radius_spw = spw_radius * (i + 1) / n_rows
                angle_spw = 2* np.pi * (j+1) / n_cols
        
                dx = radius_spw * np.sin(angle_spw)/1000
                dy = radius_spw * np.cos(angle_spw)/1000
                dx /= (R * np.cos(y_spw_eye * np.pi/180)) * 2 * np.pi/360
                dy /= R * 2 * np.pi/360
                x = x_spw_eye + dx
                y = y_spw_eye + dy
                # 使用正则表达式匹配第4个数
                match_u = re.findall(r'\d+\.\d+', rows[4+i][0])
                if len(match_u) >= j+1:
                    u = float(match_u[j])
                else:
                    raise ValueError
                match_direction = re.findall(r'\d+\.\d+', rows[4+500+i][0])
                if len(match_direction) >= j+1:
                    direction = float(match_direction[j])
                else:
                    raise ValueError
                match_pressure = re.findall(r'\d+\.\d+', rows[4+500*2+i][0])
                if len(match_pressure) >= j+1:
                    pressure = float(match_pressure[j])
                else:
                    raise ValueError
                ux = - u * np.sin(direction * np.pi/180)
                uy = - u * np.cos(direction * np.pi/180)
                new_row = {
                    'x': x, 
                    'y': y, 
                    't': t, 
                    'u': u, 
                    'direction': direction, 
                    'ux': ux,
                    'uy': uy,
                    'pressure': pressure
                }
                data_9711.append(new_row)

## 创建规则网络

In [16]:
# Create regular grid
num_points = int(np.sqrt(n_cols * (200/10)))
lat = np.linspace(min([row['y'] for row in data_9711]),max([row['y'] for row in data_9711]), num_points)
lon = np.linspace(min([row['x'] for row in data_9711]), max([row['x'] for row in data_9711]), num_points)
y_grid, x_grid = np.meshgrid(lat, lon, indexing='ij')


datasets_9711_u = []
datasets_9711_pressure = []
for time in set([row['t'] for row in data_9711]):
    rows = [row for row in data_9711 if row['t'] == time]
    
    # interpolate ux
    ux_data = np.array([row['ux'] for row in rows])
    ux_interpolate = griddata((np.array([row['y'] for row in rows]), np.array([row['x'] for row in rows])), ux_data, (y_grid, x_grid), method='linear')
    
    # interpolate uy
    uy_data = np.array([row['uy'] for row in rows])
    uy_interpolate = griddata((np.array([row['y'] for row in rows]), np.array([row['x'] for row in rows])), uy_data, (y_grid, x_grid), method='linear')
    
    # interpolate pressure
    pressure_data = np.array([row['pressure'] for row in rows])
    pressure_interpolate = griddata((np.array([row['y'] for row in rows]), np.array([row['x'] for row in rows])), pressure_data, (y_grid, x_grid), method='linear')
    
    # 创建 netCDF 文件并添加变量
    ds_u = xr.Dataset(
        {
            'ux': (("lat","lon"), ux_interpolate),
            'uy': (("lat","lon"), uy_interpolate)
        },
        coords={
            "lat": lat,
            "lon": lon,
            "time": time,
        },
        attrs={
            "title": "Interpolated data",
            "description": "Interpolated data for ux, uy, and pressure",
            "history": "Created by Bruce"
        }
    )
    
    
    ds_pressure = xr.Dataset(
        {
            'pressure_interpolate': (("lat","lon"), pressure_interpolate)
        },
        coords={
            "lat": lat,
            "lon": lon,
            "time": time,
        },
        attrs={
            "title": "Interpolated data",
            "description": "Interpolated data for ux, uy, and pressure",
            "history": "Created by Bruce"
        }
    )
    
    
    datasets_9711_u.append(ds_u)
    datasets_9711_pressure.append(ds_pressure)

# 合并数据集
combined_dataset_u = xr.concat(datasets_9711_u, dim="time").sortby("time")
combined_dataset_pressure = xr.concat(datasets_9711_pressure, dim="time").sortby("time")

# 将数据写入 netCDF 文件
combined_dataset_u.to_netcdf("../data/intermediate/interpolated_data_u.nc")
combined_dataset_pressure.to_netcdf("../data/intermediate/interpolated_data_pressure.nc")

# 淹没

In [3]:
test_map_2050 = xr.open_dataset("../data/raw/test_map_2050.nc")

# 坐标信息
map_2050_variables ={
    'x_coordinate': test_map_2050.mesh2d_face_x.values,
    'y_coordinate': test_map_2050.mesh2d_face_y.values,
    'time_data': test_map_2050.time.values,
    'water_level': test_map_2050.mesh2d_waterdepth.values,
}

# Create regular grid
num_points = int(np.sqrt(len(map_2050_variables['x_coordinate'])))
lat = np.linspace(map_2050_variables['y_coordinate'].min(), map_2050_variables['y_coordinate'].max(), num_points)
lon = np.linspace(map_2050_variables['x_coordinate'].min(), map_2050_variables['x_coordinate'].max(), num_points)
y_grid, x_grid = np.meshgrid(lat, lon, indexing='ij')

# 把坐标信息打包在一起
coordinate_data = {
    'y_coordinate': map_2050_variables['y_coordinate'],
    'x_coordinate': map_2050_variables['x_coordinate'],
    'lat': lat,
    'lon': lon,
    'y_grid': y_grid,
    'x_grid': x_grid,
}


In [5]:
def apply_restricted_area_include(dataset, restricted_area_path):
    # 获取数据集的变量名
    variable_names = list(dataset.data_vars.keys())

    # 提取 xarray 数据的坐标和值
    lats = dataset['lat'].values
    lons = dataset['lon'].values

    # 将坐标整理为一个 Pandas 数据框
    df_coords = pd.DataFrame({'lat': np.repeat(lats, len(lons)),
                              'lon': np.tile(lons, len(lats))})

    # 将 Pandas 数据框转换为 GeoPandas 数据框
    geometry = [Point(xy) for xy in zip(df_coords['lon'], df_coords['lat'])]
    gdf_coords = gpd.GeoDataFrame(df_coords, crs="EPSG:4326", geometry=geometry)

    # 加载禁区范围 shapefile
    restricted_area = gpd.read_file(restricted_area_path)

    # 使用空间连接找到在禁区范围内的点
    gdf_within_restricted_area = gpd.sjoin(gdf_coords, restricted_area, how='left', predicate='within')
    restricted_indices = gdf_within_restricted_area[gdf_within_restricted_area['index_right'].notnull()].index

    for variable_name in variable_names:
        # 提取数据变量的值
        values = dataset[variable_name].values

        # 将值整理为一个 Pandas 数据框
        df_values = pd.DataFrame({'value': values.flatten()})

        # 将不在禁区范围内的数据以及小于0.15的数据设置为 nan
        df_values.loc[(~df_values.index.isin(restricted_indices)), 'value'] = np.nan

        # 获取数据变量的原始形状
        original_shape = dataset[variable_name].shape

        # 更新数据集中的禁区范围内的数据
        dataset[variable_name].values = df_values['value'].values.reshape(original_shape)

    return dataset

In [7]:
shanghai = "../data/raw/shanghai_shapefile/shanghai.shp"
# 定义插值函数
def generate_output_scaler_time_mesh2d(variable, variable_name, title, description, output_filename):
    datasets = []
    for time_index, time in enumerate(map_2050_variables['time_data']):
        # 对数据进行插值
        variable_regular_data = griddata((coordinate_data['y_coordinate'], coordinate_data['x_coordinate']), variable[time_index, :], (y_grid, x_grid), method='linear')

        # 创建 netCDF 文件并添加变量
        ds = xr.Dataset(
        {
            variable_name: (("lat","lon"), variable_regular_data)
        },
        coords={
            "lat": lat,
            "lon": lon,
            "time": time,
        },
        attrs={
            "title": title,
            "description": description,
            "history": "Created by Bruce"}
        )
        ds = apply_restricted_area_include(ds,shanghai)
        print('yes')
        datasets.append(ds)

    # 合并数据集
    combined_dataset = xr.concat(datasets, dim="time")
    
    # 将数据写入 netCDF 文件
    combined_dataset.to_netcdf(f"../data/intermediate/"+ output_filename)

In [8]:
## generate_output_vector_time_mesh2d 标量网格插值，坐标包含mesh2d和time
generate_output_scaler_time_mesh2d(map_2050_variables['water_level'], "mesh2d_waterdepth", "Water_level", "Water_level", "Water_level_inter.nc")

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [6]:
import matplotlib.colors as mcolors

water_level = xr.open_dataset("../data/intermediate/water_level_h.nc")
water_level = water_level.where((water_level <= 1.2) | np.isnan(water_level), 1.2)
water_level = water_level.where(water_level >= 0.15, np.nan)


# 创建一个从黄色到红色的颜色映射
cmap = mcolors.LinearSegmentedColormap.from_list("mycmap", ["yellow", "red"])

def create_contour_images(mesh_lon, mesh_lat, variable, output_dir, t=None):
    if t is None:
        data_t = variable.values
    else:
        data_t = variable.values[t, :, :]

    fig, ax = plt.subplots()
    cs = ax.contourf(mesh_lon, mesh_lat, data_t, cmap=cmap)  # 使用新的颜色映射

    # 隐藏坐标轴
    ax.set_axis_off()

    # 检查文件夹是否存在，如果不存在则创建
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # 将等值线图保存为 PNG 图片
    if t is None:
        plt.savefig(f"{output_dir}/{output_dir.split('/')[-1]}.png", dpi=300, transparent=True, bbox_inches="tight", pad_inches=0)
    else:
        plt.savefig(f"{output_dir}/{output_dir.split('/')[-1]}_{t}.png", dpi=300, transparent=True, bbox_inches="tight", pad_inches=0)
    plt.close(fig)


# 为每个时间步长创建等值线
mesh_lon, mesh_lat = np.meshgrid(lon, lat)
time_index = water_level.coords['time'].values

# 计算边界位置，来获得SW和NE方位角的（lat,lon）数据
sw_bound = (np.min(mesh_lat), np.min(mesh_lon))
ne_bound = (np.max(mesh_lat), np.max(mesh_lon))


# 创建其他变量的等值线图并保存为 PNG 图片
variables_output_dir = {
    'Water_level': "../figures/Water_level",
}

variables = {
    'Water_level': water_level.mesh2d_,
}

for key in variables_output_dir:
    for t in range(time_index.shape[0]):
        create_contour_images(mesh_lon, mesh_lat, variables[key], variables_output_dir[key], t)


# 将边界数据写入csv文件
with open('../data/intermediate/boundary_data.csv', mode='w', newline='') as csv_file:
    fieldnames = ['Boundary', 'Latitude', 'Longitude']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    # 将sw_bound和ne_bound数据写入csv文件
    writer.writerow({'Boundary': 'SW', 'Latitude': sw_bound[0], 'Longitude': sw_bound[1]})
    writer.writerow({'Boundary': 'NE', 'Latitude': ne_bound[0], 'Longitude': ne_bound[1]})

In [ ]:
import numpy as np
import pyproj
import xarray as xr
import rasterio

with rasterio.open("../data/raw/shanghai_shapefile/DEM.tif") as dem:
    dem_array = dem.read(1).astype('float64')
    dem.close()

# 从 DEM 文件中获取坐标信息
transform = dem.transform
nx, ny = dem.width, dem.height

# 创建 lon 和 lat 坐标
x, _ = transform * (np.arange(nx)+0.5, np.array([0]*nx)+0.5)
_, y = transform * (np.array([0]*ny)+0.5, np.arange(ny)+0.5)

# 创建网格
xx, yy = np.meshgrid(x, y)

# 定义转换器
transformer = pyproj.Transformer.from_crs("EPSG:32651", "EPSG:4326")

# 将 DEM 数据的坐标转换为 WGS84
lat,lon  = transformer.transform(xx, yy)

lat = lat[:,0]
lon = lon[0,:]
# 将 dem_array 转换为 xarray DataArray
dem_data = xr.DataArray(
    dem_array,
    coords={"lat": lat[::-1], "lon": lon},  # 注意我们翻转了y坐标，因为通常raster数据的原点在左上角
    dims=["lat", "lon"],
)

water_level = xr.open_dataset("../data/intermediate/Water_level_inter.nc")
# 将 DEM 数据集的坐标插值为与水位数据集的坐标精度相同
dem_data_interp = dem_data.interp(lat=water_level.lat, lon=water_level.lon)

# 降雨

In [ ]:
rainfall = xr.open_dataset("../data/raw/rainfall_9711WRF_2050.nc")
def create_contour_images(mesh_lon, mesh_lat, variable, output_dir, t=None):
    if t is None:
        data_t = variable.values
    else:
        data_t = variable.values[t, :, :]

    fig, ax = plt.subplots()
    cs = ax.contourf(mesh_lon, mesh_lat, data_t, cmap='viridis')

    # 隐藏坐标轴
    ax.set_axis_off()

    # 检查文件夹是否存在，如果不存在则创建
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # 将等值线图保存为 PNG 图片
    if t is None:
        plt.savefig(f"{output_dir}/{output_dir.split('/')[-1]}.png", dpi=300, transparent=True, bbox_inches="tight", pad_inches=0)
    else:
        plt.savefig(f"{output_dir}/{output_dir.split('/')[-1]}_{t}.png", dpi=300, transparent=True, bbox_inches="tight", pad_inches=0)
    plt.close(fig)


# 为每个时间步长创建等值线
mesh_lon = rainfall.longitude.values
mesh_lat = rainfall.latitude.values
time_index = rainfall.coords['time'].values

# 计算边界位置，来获得SW和NE方位角的（lat,lon）数据
sw_bound = (np.min(mesh_lat), np.min(mesh_lon))
ne_bound = (np.max(mesh_lat), np.max(mesh_lon))


# 创建其他变量的等值线图并保存为 PNG 图片
variables_output_dir = {
    'rainfall': "../data/intermediate/Rainfall",
}

variables = {
    'rainfall': rainfall.rainfall,
}

for key in variables_output_dir:
    for t in range(time_index.shape[0]):
        create_contour_images(mesh_lon, mesh_lat, variables[key], variables_output_dir[key], t)


# 将边界数据写入csv文件
with open('../data/intermediate/rainfall_boundary_data.csv', mode='w', newline='') as csv_file:
    fieldnames = ['Boundary', 'Latitude', 'Longitude']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    # 将sw_bound和ne_bound数据写入csv文件
    writer.writerow({'Boundary': 'SW', 'Latitude': sw_bound[0], 'Longitude': sw_bound[1]})
    writer.writerow({'Boundary': 'NE', 'Latitude': ne_bound[0], 'Longitude': ne_bound[1]})

# 潮位

In [7]:
FlowFM_map = xr.open_dataset("../data/raw/FlowFM_map.nc")

# 坐标信息
map_2050_variables ={
    'x_coordinate': FlowFM_map.mesh2d_face_x.values,
    'y_coordinate': FlowFM_map.mesh2d_face_y.values,
    'time_data': FlowFM_map.time.values,
    'water_level': FlowFM_map.mesh2d_s1.values,
}

# Create regular grid
num_points = int(np.sqrt(len(map_2050_variables['x_coordinate'])))
lat = np.linspace(map_2050_variables['y_coordinate'].min(), map_2050_variables['y_coordinate'].max(), num_points)
lon = np.linspace(map_2050_variables['x_coordinate'].min(), map_2050_variables['x_coordinate'].max(), num_points)
y_grid, x_grid = np.meshgrid(lat, lon, indexing='ij')

# 把坐标信息打包在一起
coordinate_data = {
    'y_coordinate': map_2050_variables['y_coordinate'],
    'x_coordinate': map_2050_variables['x_coordinate'],
    'lat': lat,
    'lon': lon,
    'y_grid': y_grid,
    'x_grid': x_grid
}

In [133]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

def apply_restricted_area_exclude(dataset, restricted_area_path):
    # 获取数据集的变量名
    variable_names = list(dataset.data_vars.keys())

    # 提取 xarray 数据的坐标和值
    lats = dataset['lat'].values
    lons = dataset['lon'].values

    # 将坐标整理为一个 Pandas 数据框
    df_coords = pd.DataFrame({'lat': np.repeat(lats, len(lons)),
                              'lon': np.tile(lons, len(lats))})

    # 将 Pandas 数据框转换为 GeoPandas 数据框
    geometry = [Point(xy) for xy in zip(df_coords['lon'], df_coords['lat'])]
    gdf_coords = gpd.GeoDataFrame(df_coords, crs="EPSG:4326", geometry=geometry)

    # 加载禁区范围 shapefile
    restricted_area = gpd.read_file(restricted_area_path)

    # 使用空间连接找到在禁区范围内的点
    gdf_within_restricted_area = gpd.sjoin(gdf_coords, restricted_area, how='inner', predicate='within')

    for variable_name in variable_names:
        # 提取数据变量的值
        values = dataset[variable_name].values.flatten()

        # 获取数据变量的原始形状
        original_shape = dataset[variable_name].shape

        # 创建一个数组，填充原始数据
        values_filled = np.array(values)

        # 将禁区范围内的数据设置为 nan
        values_filled[gdf_within_restricted_area.index] = np.nan

        # 更新数据集中的禁区范围内的数据
        dataset[variable_name].values = values_filled.reshape(original_shape)

    return dataset


In [134]:
China = '../data/raw/China_shapefile/country.shp'
# 定义插值函数
def generate_output_scaler_time_mesh2d(variable, variable_name, title, description, output_filename):
    datasets = []
    for time_index, time in enumerate(map_2050_variables['time_data']):
        # 对数据进行插值
        variable_regular_data = griddata((coordinate_data['y_coordinate'], coordinate_data['x_coordinate']), variable[time_index, :], (y_grid, x_grid), method='linear')

        # 创建 netCDF 文件并添加变量
        ds = xr.Dataset(
        {
            variable_name: (("lat","lon"), variable_regular_data)
        },
        coords={
            "lat": lat,
            "lon": lon,
            "time": time,
        },
        attrs={
            "title": title,
            "description": description,
            "history": "Created by Bruce"}
        )
        ds = apply_restricted_area_exclude(ds, China)
        print('yes')
        datasets.append(ds)

    # 合并数据集
    combined_dataset = xr.concat(datasets, dim="time")
    
    # 将数据写入 netCDF 文件
    combined_dataset.to_netcdf(f"../data/intermediate/"+ output_filename)


In [136]:
## generate_output_vector_time_mesh2d 标量网格插值，坐标包含mesh2d和time
generate_output_scaler_time_mesh2d(map_2050_variables['water_level'], "mesh2d_s1", "Water_level", "Water_level", "Tide_level_inter.nc")

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [8]:
import matplotlib.colors as mcolors
water_level = xr.open_dataset("../data/intermediate/tide_level_h.nc")
water_level = water_level.sel(lat=slice(30, 32.5), lon=slice(None, 123.5))
water_level = water_level.where((water_level <= 2) | np.isnan(water_level), 2)
water_level = water_level.where((water_level >= -2) | np.isnan(water_level), -2)

lon = water_level.lon.values
lat = water_level.lat.values
cmap = mcolors.LinearSegmentedColormap.from_list("mycmap", ["white", "darkblue"], N=256)

def create_contour_images(mesh_lon, mesh_lat, variable, output_dir, t=None):
    if t is None:
        data_t = variable.values
    else:
        data_t = variable.values[t, :, :]

    fig, ax = plt.subplots()
    cs = ax.contourf(mesh_lon, mesh_lat, data_t, cmap=cmap)  # 使用新的颜色映射

    # 隐藏坐标轴
    ax.set_axis_off()

    # 检查文件夹是否存在，如果不存在则创建
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # 将等值线图保存为 PNG 图片
    if t is None:
        plt.savefig(f"{output_dir}/{output_dir.split('/')[-1]}.png", dpi=300, transparent=True, bbox_inches="tight", pad_inches=0)
    else:
        plt.savefig(f"{output_dir}/{output_dir.split('/')[-1]}_{t}.png", dpi=300, transparent=True, bbox_inches="tight", pad_inches=0)
    plt.close(fig)


# 为每个时间步长创建等值线
mesh_lon, mesh_lat = np.meshgrid(lon, lat)
time_index = water_level.coords['time'].values

# 计算边界位置，来获得SW和NE方位角的（lat,lon）数据
sw_bound = (np.min(mesh_lat), np.min(mesh_lon))
ne_bound = (np.max(mesh_lat), np.max(mesh_lon))


# 创建其他变量的等值线图并保存为 PNG 图片
variables_output_dir = {
    'Tide_level': "../figures/Tide_level",
}

variables = {
    'Tide_level': water_level.mesh2d_s1,
}

for key in variables_output_dir:
    for t in range(time_index.shape[0]):
        create_contour_images(mesh_lon, mesh_lat, variables[key], variables_output_dir[key], t)


# 将边界数据写入csv文件
with open('../data/intermediate/Tide_boundary_data.csv', mode='w', newline='') as csv_file:
    fieldnames = ['Boundary', 'Latitude', 'Longitude']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    # 将sw_bound和ne_bound数据写入csv文件
    writer.writerow({'Boundary': 'SW', 'Latitude': sw_bound[0], 'Longitude': sw_bound[1]})
    writer.writerow({'Boundary': 'NE', 'Latitude': ne_bound[0], 'Longitude': ne_bound[1]})

# 地形切割

In [ ]:
def apply_restricted_area_include(dataset, restricted_area_path):
    # 获取数据集的变量名
    variable_names = list(dataset.data_vars.keys())

    # 提取 xarray 数据的坐标和值
    lats = dataset['lat'].values
    lons = dataset['lon'].values

    # 将坐标整理为一个 Pandas 数据框
    df_coords = pd.DataFrame({'lat': np.repeat(lats, len(lons)),
                              'lon': np.tile(lons, len(lats))})

    # 将 Pandas 数据框转换为 GeoPandas 数据框
    geometry = [Point(xy) for xy in zip(df_coords['lon'], df_coords['lat'])]
    gdf_coords = gpd.GeoDataFrame(df_coords, crs="EPSG:4326", geometry=geometry)

    # 加载禁区范围 shapefile
    restricted_area = gpd.read_file(restricted_area_path)

    # 使用空间连接找到在禁区范围内的点
    gdf_within_restricted_area = gpd.sjoin(gdf_coords, restricted_area, how='left', predicate='within')
    restricted_indices = gdf_within_restricted_area[gdf_within_restricted_area['index_right'].notnull()].index

    for variable_name in variable_names:
        # 提取数据变量的值
        values = dataset[variable_name].values

        # 将值整理为一个 Pandas 数据框
        df_values = pd.DataFrame({'value': values.flatten()})

        # 将不在禁区范围内的数据设置为 nan
        df_values.loc[~df_values.index.isin(restricted_indices), 'value'] = np.nan

        # 获取数据变量的原始形状
        original_shape = dataset[variable_name].shape

        # 更新数据集中的禁区范围内的数据
        dataset[variable_name].values = df_values['value'].values.reshape(original_shape)

    return dataset

In [ ]:

water_level = xr.open_dataset("../data/intermediate/Water_level_inter.nc")
# rainfall = xr.open_dataset('../data/raw/rainfall_9711WRF_2050.nc')
tide_level = xr.open_dataset('../data/intermediate/Tide_level_inter.nc')

shanghai = "../data/raw/shanghai_shapefile/Shanghai-2020.shp"
water_level = apply_restricted_area(water_level, shanghai)
# rainfall = apply_restricted_area(rainfall, shanghai)

China = '../data/raw/China_shapefile/bou1_4p.shp'
tide_level = apply_restricted_area(tide_level, China)